# 1. Introduction

### Topic

For this Capstone Project - The Battle of Neighborhoods, I am planning to compre the similarity and dissmilarity of two cities in US, from various aspects-such as cost of living, amenity, etc.


### Personal Background

I am working as data analyst at Dallas, Texas. Due to some family reasons, I am planning move to Boston, Massachusetts in the near future (The first time live in another state). Currently, I am actively seeking jobs and interviewing with some companyies. 

### Why this topic

Texas is such a wonderful place for living: No State income Tax and low cost of living. However, Boston is such a pricy city: State income Tax and high cost of living. I am hoping my project could provide some basic guildline on hwo to compare the quality of life in two different cities


# 2.Data section

### Dataset would be used

    1. Foursquare data of Dallas with specific address
    2. Foursquare data of Boston with specific address
    3. Rental Cost from Zillow
    
These Foursquare data will be used for compare the living similariy (standard of living = level of happieness). Cost of living will be used to evalute the monthly cost of living in both address (which can be used to find the leverage of the salray matching ratio).


# 3. Methodology

The frist part focus on the exploratory analysis in dallas area. The address I am using is **4601 Amesbury drive**. I have personally live in this area for more than 2 years. 

The raidus is set to 3000 meters (bicycle distance). In this area, I will give a list of information about the resturant, fitness center, gas station, and grocery store.

In [1]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')


usage: conda-script.py [-h] [-V] command ...
conda-script.py: error: unrecognized arguments: # uncomment this line if you haven't completed the Foursquare API lab


Libraries imported.


### Dallas 

In [2]:
# Shenton Way, District 01, Singapore
address = '4601 Amesbury Dr, Dallas'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dallas are {}, {}.'.format(latitude, longitude))

C:\Users\micha\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Dallas are 32.8425069669687, -96.7639288225917.


In [3]:
neighborhood_latitude=latitude
neighborhood_longitude=longitude

CLIENT_ID = 'JMVDMC0MO0U3MKR5NL3SU3GFHT1TOLO4FYQE52DUHCCISMRW' # your Foursquare ID
CLIENT_SECRET = 'SBACRDKIXOBOXSYMYZIIKHKIJLXAR1LX2DAK3BX12KTNJQNK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [49]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 3000 

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Velvet Taco,New American Restaurant,32.844704,-96.769743
1,Torchy's Tacos,Taco Place,32.841805,-96.771375
2,Central Market,Grocery Store,32.849531,-96.768632
3,Grub Burger Bar,Burger Joint,32.846878,-96.770289
4,Starbucks,Coffee Shop,32.847615,-96.769736
5,Raising Cane's Chicken Fingers,Fried Chicken Joint,32.848603,-96.769570
6,Smoothie King,Smoothie Shop,32.836285,-96.769834
7,Twisted Root,Burger Joint,32.841991,-96.772032
8,Campisi's Restaurant - The Egyptian Lounge,Italian Restaurant,32.836311,-96.771643
9,Kostas Cafe,Greek Restaurant,32.847321,-96.769895


In [31]:
# The unique labor of categories
print(nearby_venues['categories'].unique())

['New American Restaurant' 'Taco Place' 'Grocery Store' 'Burger Joint'
 'Coffee Shop' 'Fried Chicken Joint' 'Smoothie Shop' 'Italian Restaurant'
 'Greek Restaurant' 'Diner' 'Indie Movie Theater' 'Thai Restaurant'
 'Seafood Restaurant' 'Bar' 'Shopping Plaza' 'Clothing Store'
 'Vietnamese Restaurant' 'Latin American Restaurant'
 'Furniture / Home Store' 'Music Venue' 'Salon / Barbershop'
 'Gym / Fitness Center' 'Irish Pub' 'Donut Shop' 'Mexican Restaurant'
 'Health Food Store' 'Yoga Studio' 'Library' 'Steakhouse' 'Hotel'
 'Wings Joint' 'Pizza Place' 'Sporting Goods Shop' 'Gym' 'Beer Garden'
 'Ice Cream Shop' 'Spa' 'Sushi Restaurant' 'Sandwich Place' 'Theater'
 'Tailor Shop' 'Pet Store' 'Video Store' 'Trail' 'Restaurant'
 'American Restaurant' 'Frozen Yogurt Shop' 'Bistro' 'Supermarket'
 'Fast Food Restaurant' 'Toy / Game Store' 'French Restaurant'
 'Liquor Store' 'Juice Bar' 'Dance Studio' 'Park' 'Playground' 'Bakery'
 'Cupcake Shop' 'Pharmacy' 'Art Museum' 'Performing Arts Venue']


Since I am only interseted in Food place, Shopping place, Gym, and Grocery. Thus, I will convert those multiple categories to 5 simply list.

- Food (Restaurant, fast food, etc.)

- Gym (Gym, Yoga, etc.)

- Shooping (Shopping center, furniture shop, etc.)

- Grocery (Grocery, Supermakrt, etc.)

- Other (rest categories)

In [51]:
temp=nearby_venues

food_list=['restaurant', 'taco', 'burger', 'chicken', 'diner', 'steakhouse', 'wings', 'pizza', 'sandwich']
gym_list=['gym', 'fitness', 'yoga']
shop_list=['shopping', 'clothing',' furniture', 'goods']
grocery_list=['grocery']


for cate in temp['categories'].unique():
    count=0
    # Change the category to food
    if count!=1:
        for item in food_list:
            if item in cate.lower(): # If the key word in the categroy, then we replace the category to food
                count=1
                temp['categories'].replace(cate,'Food',inplace=True)
                
    if count!=1:
        for item in gym_list:
            if item in cate.lower(): # If the key word in the categroy, then we replace the category to gym
                count=1
                temp['categories'].replace(cate,'Gym',inplace=True)
                
    if count!=1:
        for item in shop_list:
            if item in cate.lower(): # If the key word in the categroy, then we replace the category to shopping
                count=1
                temp['categories'].replace(cate,'Shopping',inplace=True)
                
    if count!=1:
        for item in grocery_list:
            if item in cate.lower(): # If the key word in the categroy, then we replace the category to grocery
                count=1
                temp['categories'].replace(cate,'Grocery',inplace=True)
                
    if count==0:
        temp['categories'].replace(cate,'Other',inplace=True)
        
df_dallas=temp

df_dallas.head(10)

,name,categories,lat,lng
0,Velvet Taco,Food,32.844704,-96.769743
1,Torchy's Tacos,Food,32.841805,-96.771375
2,Central Market,Grocery,32.849531,-96.768632
3,Grub Burger Bar,Food,32.846878,-96.770289
4,Starbucks,Other,32.847615,-96.769736
5,Raising Cane's Chicken Fingers,Food,32.848603,-96.769570
6,Smoothie King,Other,32.836285,-96.769834
7,Twisted Root,Food,32.841991,-96.772032
8,Campisi's Restaurant - The Egyptian Lounge,Food,32.836311,-96.771643
9,Kostas Cafe,Food,32.847321,-96.769895


In [63]:
# Count each Category

for item in df_dallas['categories'].unique():
    print(item, ": ", df_dallas[df_dallas['categories']==item].shape[0])

Food :  49
Grocery :  1
Other :  41
Shopping :  3
Gym :  6


In [55]:
temp=df_dallas


map_temp = folium.Map(location=[latitude, longitude], zoom_start=18)
# add markers to map
for lat, lng, label, cate in zip(temp['lat'], temp['lng'], temp['name'],temp['categories']):
    label = folium.Popup(label, parse_html=True)
    
    folium.RegularPolygonMarker(
    [latitude,longitude],
    radius=5,
    popup='Home',
    color='Red',
    fill_color='#0f0f0f',
    fill_opacity=1,
    ).add_to(map_temp)
    
    if cate=='Food':
        folium.RegularPolygonMarker(
            [lat, lng],
            number_of_sides=10,
            radius=3,
            popup=label,
            color='black',
            fill_color='#0f0f0f',
            fill_opacity=1,
        ).add_to(map_temp)  
    
    
    if cate=='Gym':
        folium.RegularPolygonMarker(
            [lat, lng],
            number_of_sides=10,
            radius=3,
            popup=label,
            color='yellow',
            fill_color='#0f0f0f',
            fill_opacity=1,
        ).add_to(map_temp)
    
    if cate=='Shopping':
        folium.RegularPolygonMarker(
            [lat, lng],
            number_of_sides=10,
            radius=3,
            popup=label,
            color='blue',
            fill_color='#0f0f0f',
            fill_opacity=1,
        ).add_to(map_temp)  
        
    if cate=='Grocery':
        folium.RegularPolygonMarker(
            [lat, lng],
            number_of_sides=10,
            radius=3,
            popup=label,
            color='green',
            fill_color='#0f0f0f',
            fill_opacity=1,
        ).add_to(map_temp)  


        map_temp

There are numerous food place surrounded this apartment complex (black dot)

### Boston

I am really interested on this following apartment complex in Revere since it has public transpotation (train) and a beach nearby, with the access to a nicely built apartment's Gym.

Here is the webpage of this complex: https://www.onebeachmont.com/


In [56]:
# Shenton Way, District 01, Singapore
address = '205 Revere Beach Pkwy, Revere'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Dallas are {}, {}.'.format(latitude, longitude))

C:\Users\micha\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  after removing the cwd from sys.path.


The geograpical coordinate of Dallas are 42.39989905, -70.9941980387861.


In [66]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 3000 
neighborhood_latitude=latitude
neighborhood_longitude=longitude
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)

results = requests.get(url).json()

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Dimino's Subs,Sandwich Place,42.399133,-70.994174
1,Flaming's Buffet,Chinese Restaurant,42.397576,-71.000588
2,Nick's Place,Pizza Place,42.405731,-70.991499
3,Twist & Shake,Ice Cream Shop,42.406883,-70.991513
4,"sayar food market, inc",Grocery Store,42.410147,-71.000876
5,BK's Bar and Grill,Bar,42.407173,-70.992199
6,Peter's Super Beef,American Restaurant,42.408801,-70.993785
7,Revere Beach,Beach,42.408507,-70.991526
8,Dollar Tree,Discount Store,42.397448,-71.000572
9,The Bagel Bin Deli & Restaurant,Diner,42.407245,-70.993037


In [67]:
nearby_venues['categories'].unique()

array(['Sandwich Place', 'Chinese Restaurant', 'Pizza Place',
       'Ice Cream Shop', 'Grocery Store', 'Bar', 'American Restaurant',
       'Beach', 'Discount Store', 'Diner', 'Bakery', 'Mexican Restaurant',
       'Seafood Restaurant', 'Circus', 'Italian Restaurant', 'Restaurant',
       'Coffee Shop', 'Breakfast Spot', 'Café', 'Big Box Store',
       'Gas Station', 'Donut Shop', 'Hotel', 'BBQ Joint',
       'Gym / Fitness Center', 'Supplement Shop', 'Paintball Field',
       'Video Game Store', 'Shoe Store', 'Liquor Store', 'Supermarket',
       'New American Restaurant', 'Tex-Mex Restaurant', 'Deli / Bodega',
       'Pharmacy', 'Department Store', 'Buffet', 'Mobile Phone Shop',
       'Auto Garage', 'Asian Restaurant', 'Convenience Store',
       'Vietnamese Restaurant', 'Rental Car Location', 'Bagel Shop',
       "Men's Store", 'Pub', 'Bank', 'Warehouse Store', 'Cosmetics Shop'],
      dtype=object)

In [59]:
temp=nearby_venues

food_list=['restaurant', 'taco', 'burger', 'chicken', 'diner', 'steakhouse', 'wings', 'pizza', 'sandwich']
gym_list=['gym', 'fitness', 'yoga']
shop_list=['shopping', 'clothing',' furniture', 'goods']
grocery_list=['grocery']


for cate in temp['categories'].unique():
    count=0
    # Change the category to food
    if count!=1:
        for item in food_list:
            if item in cate.lower(): # If the key word in the categroy, then we replace the category to food
                count=1
                temp['categories'].replace(cate,'Food',inplace=True)
                
    if count!=1:
        for item in gym_list:
            if item in cate.lower(): # If the key word in the categroy, then we replace the category to gym
                count=1
                temp['categories'].replace(cate,'Gym',inplace=True)
                
    if count!=1:
        for item in shop_list:
            if item in cate.lower(): # If the key word in the categroy, then we replace the category to shopping
                count=1
                temp['categories'].replace(cate,'Shopping',inplace=True)
                
    if count!=1:
        for item in grocery_list:
            if item in cate.lower(): # If the key word in the categroy, then we replace the category to grocery
                count=1
                temp['categories'].replace(cate,'Grocery',inplace=True)
                
    if count==0:
        temp['categories'].replace(cate,'Other',inplace=True)
        
df_revere=temp

df_revere.head(10)

,name,categories,lat,lng
0,Dimino's Subs,Food,42.399133,-70.994174
1,Flaming's Buffet,Food,42.397576,-71.000588
2,Nick's Place,Food,42.405731,-70.991499
3,Twist & Shake,Other,42.406883,-70.991513
4,"sayar food market, inc",Grocery,42.410147,-71.000876
5,BK's Bar and Grill,Other,42.407173,-70.992199
6,Peter's Super Beef,Food,42.408801,-70.993785
7,Revere Beach,Other,42.408507,-70.991526
8,Dollar Tree,Other,42.397448,-71.000572
9,The Bagel Bin Deli & Restaurant,Food,42.407245,-70.993037


In [64]:
# Count each Category

for item in df_revere['categories'].unique():
    print(item, ": ", df_revere[df_revere['categories']==item].shape[0])

Food :  33
Other :  64
Grocery :  2
Gym :  1


In [60]:
temp=df_revere


map_temp = folium.Map(location=[latitude, longitude], zoom_start=18)
# add markers to map
for lat, lng, label, cate in zip(temp['lat'], temp['lng'], temp['name'],temp['categories']):
    label = folium.Popup(label, parse_html=True)
    
    folium.RegularPolygonMarker(
    [latitude,longitude],
    radius=5,
    popup='Home',
    color='Red',
    fill_color='#0f0f0f',
    fill_opacity=1,
    ).add_to(map_temp)
    
    if cate=='Food':
        folium.RegularPolygonMarker(
            [lat, lng],
            number_of_sides=10,
            radius=3,
            popup=label,
            color='black',
            fill_color='#0f0f0f',
            fill_opacity=1,
        ).add_to(map_temp)  
    
    
    if cate=='Gym':
        folium.RegularPolygonMarker(
            [lat, lng],
            number_of_sides=10,
            radius=3,
            popup=label,
            color='yellow',
            fill_color='#0f0f0f',
            fill_opacity=1,
        ).add_to(map_temp)
    
    if cate=='Shopping':
        folium.RegularPolygonMarker(
            [lat, lng],
            number_of_sides=10,
            radius=3,
            popup=label,
            color='blue',
            fill_color='#0f0f0f',
            fill_opacity=1,
        ).add_to(map_temp)  
        
    if cate=='Grocery':
        folium.RegularPolygonMarker(
            [lat, lng],
            number_of_sides=10,
            radius=3,
            popup=label,
            color='green',
            fill_color='#0f0f0f',
            fill_opacity=1,
        ).add_to(map_temp)  

map_temp

# 4. Result

### a. Standard of living
Suppose I do not need to worry about cost of living in both area. From a quality of live perspective, the "expected" area is similar to the are I used to live at. Both areas have a decent number of eatting places with access to Gym nearly (ride a bike is an option ot the Gym). 

Good part of living in Boston:

- Beach view
- Public transportation (Train&Shuttle)

Downside of living in Boston:

- Expected high living cost
- Less enjoyable ows a car (road condition, bad weather, gas price)
- Less Basketball game (less Gym)

Overall, I will say the quality of living is around the same. Let's see how much I need to maintain this quality.

### b. Cost of living
The dataset I am using for the renting price is from Zillow's forcasting renting price. The data could be downloaded from https://www.zillow.com/research/data/ by selecting "Zillow Rent Forcast" and "All geographics (amost recent month)" under **Rental Values** Tab

In [71]:
import os

os.chdir("C:/Coursera Work/IBM Data Science")

df_rent=pd.read_csv("ZriForecast_Public.csv")

df_rent.head()

,RegionID,RegionName,RegionType,SizeRank,ZriForecast,Forecast_YoY_Change,Current_YoY_Change
0,102001,United States,Country,0,1631,0.019514,0.022843
1,394913,"New York, NY",Msa,1,2345,0.017401,0.026707
2,753899,"Los Angeles-Long Beach-Anaheim, CA",Msa,2,2649,0.013628,0.013752
3,394463,"Chicago, IL",Msa,3,1602,0.001584,0.009911
4,394514,"Dallas-Fort Worth, TX",Msa,4,1518,0.039509,0.013063


In [85]:
for place, price in zip(df_rent['RegionName'], df_rent['ZriForecast']):
    if 'Dallas' in place:
        print(place,": $",price)
        print("Annually Cost: ", price*12)
    if 'Boston' in place:
        print(place, ": $",price)
        print("Annually Cost: ", price*12)

Dallas-Fort Worth, TX : $ 1518
Annually Cost:  18216
Boston, MA : $ 2348
Annually Cost:  28176


In [86]:
44954-18216

26738

In [90]:
26738-1800*12

5138

In [91]:
5138+12*1800+28176

54914

Suppose I am making \\$55000 in dallas. After tax, I get \\$44,954 after tax. Take out the rent, I have $26,378 for spending. Suppose I am using \\$1800 for other spending (food, lesure, shopping, insurance, etc.), I can save **\\$5,138** annually.

If I am moving to Boston and I want to save the same ammount money annually, assume my monthly spending is the same besides rent, I need to make \\$54,914 after tax, which is nearly **\$75,000** before tax.

Thus, to achieve the same quality of living in Boston, I need to ask the a Salary match to \\$75,000 for a \\$55,000 annual income in Texas.

# Conclusion

The entire analysis is a basic analysis of considering Salary Match. Please be aware I have not consider the culture influence in both area, which may stronly affect the monthly cost. As we can see, we probably need to get pay 36.4% (20000/55000) in order to keep the quality of living nearly the same in Boston. 

All caculation is more like a consideration from an entire fmaily's perspective. If you were an adult who just graduated from college, you may want to think about the opportunty vesus the living quality.  